In [1]:
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras.callbacks import EarlyStopping , ReduceLROnPlateau , ModelCheckpoint
import numpy as np
from astropy.io import fits
import matplotlib.pyplot as plt
from keras.layers import Conv1D , Dropout , Flatten , MaxPooling1D, Dense, Input
from keras.layers.core import Lambda
from keras.models import Model
import pandas as pd
import random
import h5py
from sklearn.metrics import r2_score

### Descarga y orden de los datos

In [2]:
# Cargamos los datos
with h5py.File('/home/anell/Desktop/Bovy/AnellExercises/Fits_files/apogeedr14_gaiadr2_with_spectrum.h5') as F:  # ensure the file will be cleaned up
    parallax = np.array(F['parallax'])
    parallax_error = np.array(F['parallax_err'])
    spectra = np.array(F['spectra'])
    Kcorr = np.array(F['corrected_magnitude_K'])  # extinction corrected Ks
    bp_rp = np.array(F['bp_rp'])
    phot_g_mean_mag = np.array(F['phot_g_mean_mag'])
    teff = np.array(F['NN_teff'])
    apogee_id = np.array(F['APOGEE_ID'])
    snr = np.array(F['SNR'])

/home/anell/anaconda3/envs/python37-astronn/lib/python3.7/site-packages/ipykernel_launcher.py:2: H5pyDeprecationWarning: The default file mode will change to 'r' (read-only) in h5py 3.0. To suppress this warning, pass the mode you need to h5py.File(), or set the global default h5.get_config().default_file_mode, or set the environment variable H5PY_DEFAULT_READONLY=1. Available modes are: 'r', 'r+', 'w', 'w-'/'x', 'a'. See the docs for details.
  


In [3]:
len(bp_rp), parallax.shape

(60986, (60986,))

In [4]:
# Normalizamos 
phot_g_mean_mag_std = np.std(phot_g_mean_mag)
phot_g_mean_mag_mean = np.mean(phot_g_mean_mag)
norm_phot_g_mean_mag = (phot_g_mean_mag - phot_g_mean_mag_mean) / phot_g_mean_mag_std

bp_rp_std = np.std(bp_rp)
bp_rp_mean = np.mean(bp_rp)
norm_bp_rp = (bp_rp - bp_rp_mean) / bp_rp_std

teff_std = np.std(teff)
teff_mean = np.mean(teff)
norm_teff = (teff - teff_mean) / teff_std


#EStablecemos las variables que entrarán a la red y corregimos sus dimensiones
X = np.expand_dims(spectra,axis = 2)
Y = np.expand_dims(parallax,axis=1)
Y_error = np.expand_dims(parallax_error,axis=1)
K_mag = np.expand_dims(Kcorr,axis=1)
G_mag = np.expand_dims(norm_phot_g_mean_mag,axis=1)
Bp_Rp = np.expand_dims(norm_bp_rp,axis=1)
Teff = np.expand_dims(norm_teff,axis=1)
Snr = np.expand_dims(snr,axis=1)

X_offset = np.concatenate((G_mag, Bp_Rp , Teff), axis = 1) 


#Aleatorizamos la muestra
idx = []
for i in range(len(X)):
    idx.append(i)
random.seed(20)
random.shuffle(idx)

X = X[idx]                  # shape: (60986, 7514 , 1)   
Y = Y[idx]                  # shape: (60986, 1)  
K_mag = K_mag[idx]          # shape: (60986, 1) 
X_offset = X_offset[idx]    # shape: (60986, 3)
SNR = Snr[idx]              # shape: (60986, 1)

In [5]:
X.shape, Y.shape, K_mag.shape, X_offset.shape , SNR.shape

((60986, 7514, 1), (60986, 1), (60986, 1), (60986, 3), (60986, 1))

In [6]:
#Corte por buena o mala relación señal a ruido
idx_snr_good = []
idx_snr_bad = []
for i in range(len(SNR)):
    if snr[i] >= 200:
        idx_snr_good.append(i)
    else:
        idx_snr_bad.append(i)

len(idx_snr_good), len(idx_snr_bad)

(27721, 33265)

In [7]:
def size_dataset(X , Y , K_mag , X_offset , idx_snr_good, idx_snr_bad, m = None ):
    """
    INPUT:
    X.shape = (_ , 7514 , 1)
    Y.shape = (_ , 1)
    K_mag.shape = (_ , 1)
    X_offset.shape = (_ , 3)
    """
    if m == None:
        m = len(Y)
    
    X_train = np.concatenate((X[idx_snr_good][:int(0.4*m)],X[idx_snr_bad][:int(0.2*m)]),axis = 0)
    Y_train = np.concatenate((Y[idx_snr_good][:int(0.4*m)],Y[idx_snr_bad][:int(0.2*m)]),axis = 0)
    K_mag_train = np.concatenate((K_mag[idx_snr_good][:int(0.4*m)],K_mag[idx_snr_bad][:int(0.2*m)]),axis = 0)
    X_offset_train = np.concatenate((X_offset[idx_snr_good][:int(0.4*m)],X_offset[idx_snr_good][:int(0.2*m)]),axis = 0)
        
    X_val = np.concatenate((X[idx_snr_good][int(0.4*m):int(0.6*m)],X[idx_snr_bad][int(0.4*m):int(0.5*m)]),axis = 0)
    Y_val = np.concatenate((Y[idx_snr_good][int(0.4*m):int(0.6*m)],Y[idx_snr_bad][int(0.4*m):int(0.5*m)]),axis = 0)
    K_mag_val = np.concatenate((K_mag[idx_snr_good][int(0.4*m):int(0.6*m)],K_mag[idx_snr_bad][int(0.4*m):int(0.5*m)]),axis = 0)
    X_offset_val = np.concatenate((X_offset[idx_snr_good][int(0.4*m):int(0.6*m)],X_offset[idx_snr_bad][int(0.4*m):int(0.5*m)]),axis = 0)
        
    X_test = X[idx_snr_bad][int(0.5*m):int(m)]
    Y_test = Y[idx_snr_bad][int(0.5*m):int(m)]
    K_mag_test = K_mag[idx_snr_bad][int(0.5*m):int(m)]
    X_offset_test = X_offset[idx_snr_bad][int(0.5*m):int(m)]
    
    #Aleatorizamos las variables:
    idx_train = []
    for i in range(len(X_train)):
        idx_train.append(i)
    random.seed(50)
    random.shuffle(idx_train)
    
    idx_val = []
    for j in range(len(X_val)):
        idx_val.append(j)
    random.seed(100)
    random.shuffle(idx_val) 
    
    X_train = X_train[idx_train]
    Y_train = Y_train[idx_train]
    K_mag_train = K_mag_train[idx_train]
    X_offset_train = X_offset_train[idx_train]
        
    X_val = X_val[idx_val]
    Y_val = Y_val[idx_val]
    K_mag_val = K_mag_val[idx_val]
    X_offset_val = X_offset_val[idx_val]
    

    return ([X_train, Y_train, K_mag_train, X_offset_train], [X_val, Y_val, K_mag_val, X_offset_val],
            [X_test, Y_test, K_mag_test, X_offset_test])

In [8]:
#Establecemos los sets de entrenaiento, de validación y de testeo
TRAIN,VAL,TEST = size_dataset(X , Y , K_mag , X_offset , idx_snr_good, idx_snr_bad, m = 10000 )

X_train, Y_train, K_mag_train, X_offset_train = TRAIN
X_val, Y_val, K_mag_val, X_offset_val = VAL
X_test, Y_test, K_mag_test, X_offset_test = TEST

In [9]:
X_train.shape , X_val.shape , X_test.shape

((6000, 7514, 1), (3000, 7514, 1), (5000, 7514, 1))

### Generación del modelo y entrenamiento

In [24]:
def ApogeeDR14GaiaDR2(dim_t , dim_n): 
    """
    INPUT: 
    dim_t - number of time steps of spectrum 
    dim_n - number of features of spectrum
    """
    
    #SPECTRUM TO LUINOSITY
    dim_1 = 1 # number of corrected magnitude for one example 
    units = 1 #number of final output for one example
    inputs_spectra = Input(shape=(dim_t, dim_n)) 
    inputs_mag = Input(shape=(dim_1,), name="ApparentMagnitude-input")
    
    x_parallax = Conv1D(filters=4, kernel_size=3, activation='relu')(inputs_spectra)
    x_parallax = MaxPooling1D(pool_size=2)(x_parallax)
    x_parallax = Flatten()(x_parallax)
    x_parallax = Dense(128, activation='tanh')(x_parallax) 
    x_parallax = Dense(64, activation='relu')(x_parallax) 
    x_parallax = Dense(32, activation='tanh')(x_parallax)
    x_parallax = Dense(16, activation='relu')(x_parallax)
    x_parallax = Dense(units, activation='softplus')(x_parallax)
    
    outputs_parallax = Lambda(lambda function: tf.math.multiply(function[0], tf.math.pow(10., 
                              tf.math.multiply(-0.2, function[1]))),
                              name='Output-physical-parallax')([x_parallax, inputs_mag])
   
    #OFFSET CORRECTION : (optimization)
    inputs_offset = Input(shape=(3,), name="Offset-input")
    x_offset = Dense(32, activation='relu')(inputs_offset) 
    x_offset = Dense(16, activation='relu')(x_offset)
    x_offset = Dense(units, activation='tanh')(x_offset) 
    
    outputs_parallax_with_offset = Lambda(lambda function: tf.math.add(function[0], function[1]),
                                          name="Output-parallax-with-offset")([outputs_parallax, x_offset]) 
    
    #Model setup
    model =  Model(inputs = [inputs_spectra,inputs_mag, inputs_offset],outputs = [outputs_parallax_with_offset])
    
    return model 


In [25]:
n_timesteps, n_features = X_train.shape[1], X_train.shape[2]

Global_model = ApogeeDR14GaiaDR2(n_timesteps , n_features)

Global_model.summary()

Model: "functional_15"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_8 (InputLayer)            [(None, 7514, 1)]    0                                            
__________________________________________________________________________________________________
conv1d_7 (Conv1D)               (None, 7512, 4)      16          input_8[0][0]                    
__________________________________________________________________________________________________
max_pooling1d_7 (MaxPooling1D)  (None, 3756, 4)      0           conv1d_7[0][0]                   
__________________________________________________________________________________________________
flatten_7 (Flatten)             (None, 15024)        0           max_pooling1d_7[0][0]            
______________________________________________________________________________________

In [26]:
Global_model.compile(optimizer='adam', loss='mse', metrics=['mse'])

path_local = "/home/anell/Desktop/TesisAnell/Models_NN"

callbacks = [EarlyStopping(patience=10, verbose=1), ReduceLROnPlateau(factor=0.1, patience=5, min_lr=0.00001, verbose=1), 
             ModelCheckpoint(f'{path_local}/model_1.h5', verbose=1, save_best_only=True)]

Global_model.fit([X_train,K_mag_train,X_offset_train], Y_train, batch_size=256, shuffle='batch', 
                 epochs=30, callbacks=callbacks, validation_data=([X_val,K_mag_val,X_offset_val], Y_val))

Epoch 1/30
24/24 [==============================] - ETA: 0s - loss: 19.4721 - mse: 19.4721
Epoch 00001: val_loss improved from inf to 18.00375, saving model to /home/anell/Desktop/TesisAnell/Models_NN/model_1.h5
24/24 [==============================] - 31s 1s/step - loss: 19.4721 - mse: 19.4721 - val_loss: 18.0037 - val_mse: 18.0037
Epoch 2/30
24/24 [==============================] - ETA: 0s - loss: 18.3256 - mse: 18.3256
Epoch 00002: val_loss improved from 18.00375 to 17.50764, saving model to /home/anell/Desktop/TesisAnell/Models_NN/model_1.h5
24/24 [==============================] - 3s 109ms/step - loss: 18.3256 - mse: 18.3256 - val_loss: 17.5076 - val_mse: 17.5076
Epoch 3/30
23/24 [===========================>..] - ETA: 0s - loss: 17.8747 - mse: 17.8747
Epoch 00003: val_loss improved from 17.50764 to 17.39355, saving model to /home/anell/Desktop/TesisAnell/Models_NN/model_1.h5
24/24 [==============================] - 3s 109ms/step - loss: 17.6930 - mse: 17.6930 - val_loss: 17.3935 